# Submit CP2K DFT Calculation

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.code import Code
from aiida.orm import load_node 

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import nglview

from collections import OrderedDict

## Step 1: Select Structure from Database

In [ ]:
# query AiiDA database for structures
qb = QueryBuilder()
qb.append(StructureData, project=["id", "description"])
qb.order_by({StructureData: {'ctime': 'desc'}})
#print qb.count()

all_structs = OrderedDict()
all_structs["Select a Structure"] = None
for pk, descr in qb.iterall():
    label = "PK: %d"%pk
    if descr:
        label += "; descr: "+descr
    all_structs[label] = pk

def on_struct_change(c):
    global structure, atoms
    structure = load_node(drop_struct.value)
    atoms = structure.get_ase()
    for i in viewer._ngl_component_ids:
        viewer.remove_component(i)
    viewer.add_component(nglview.ASEStructure(atoms))
    viewer.add_ball_and_stick()
    viewer.add_unitcell()
    viewer.center_view()

drop_struct = widgets.Dropdown(options=all_structs, value=None)
drop_struct.observe(on_struct_change, names='value')
viewer = nglview.NGLWidget()
display(widgets.VBox([drop_struct, viewer]))

In [ ]:
%%html
<!-- fix width of descriptions -->
<style>
.widget-hbox .widget-label {width: 150px; max-width: 150px}
</style>

## Step 2: Choose parameters and test submission

In [ ]:
def get_computer_names():
    #https://github.com/aiidateam/aiida_core/blob/develop/aiida/cmdline/commands/computer.py#L1047
    qb = QueryBuilder()
    qb.append(type='computer', project=['name'])
    return zip(*qb.all())[0]

In [ ]:
l = widgets.Layout(width="400px")
inp_compname = widgets.Dropdown(description="Computer", options=get_computer_names(), layout=l)
inp_xc = widgets.Dropdown(description="XC Functional", options=["LDA", "PBE"], layout=l)
inp_basis = widgets.Dropdown(description="Basis Set", options=["SZV-MOLOPT-SR-GTH", "DZVP-MOLOPT-SR-GTH"], layout=l)
inp_descr = widgets.Text(description="Description", placeholder="(optional)", placelayout=l)
display(widgets.VBox([inp_compname,inp_xc, inp_basis, inp_descr]))

In [ ]:
def build_parameters():
    kinds = []
    for k in structure.get_kind_names():
        kinds.append({'_':k, 'basis_set': inp_basis.value,  'potential': "GTH-"+inp_xc.value})

    force_eval = {
            'method': 'Quickstep',
            'dft': {
                'basis_set_file_name': 'BASIS_MOLOPT',
                'xc': {
                    'xc_functional': {
                        '_': inp_xc.value,
                    },
                },
                
            },
            'subsys': {
                'kind': kinds,
            },
        }

    return ParameterData(dict={'force_eval':force_eval})

In [ ]:
def build_calc():
    code = Code.get_from_string("cp2k@"+inp_compname.value)

    # calc object
    calc = code.new_calc()
    calc.description = inp_descr.value

    # structure
    calc.use_structure(structure)

    # parameters
    parameters = build_parameters()
    calc.use_parameters(parameters)

    # resources
    calc.set_max_wallclock_seconds(3*60) # 3 min
    calc.set_resources({"num_machines": 1})
    return calc

## Step 3: Test submission

In [ ]:
def on_test(b):
    clear_output()
    calc = build_calc()
    calc.submit_test()
    print("Test went fine :-)")
    
btn_test = widgets.Button(description="Test")
btn_test.on_click(on_test)
display(btn_test)

## Step 4: Submit calculation

In [ ]:
def on_submit(b):
    clear_output()
    calc = build_calc()
    # store and submit
    calc.store_all()
    calc.submit()
    msg = "Submitted calculation: UUID=%s, pk=%s .<br>"%(calc.uuid,calc.dbnode.pk)
    msg += 'Results can be viewed <a href="view_cp2k_calc.ipynb?pk=%d">here</a>'%calc.dbnode.pk
    display(HTML(msg))
    
btn_submit = widgets.Button(description="Submit")
btn_submit.on_click(on_submit)
display(btn_submit)